# Day 1-00.02: 한국어 RAG 데이터 빠르게 살펴보기

오늘 사용할 한국어 RAG 데이터셋을 간단히 둘러봅니다. 셀을 위에서 아래로 실행해 구조와 통계를 파악하세요.


# neural-bridge/rag-dataset-12000 개요 & RAG 파인튜닝 안내

## 1. 데이터셋 개요 🧾

- **이름**: neural-bridge/rag-dataset-12000 
- **출처**: Hugging Face Datasets 라이브러리 
- **라이선스**: Apache-2.0 
- **크기**: 약 12,000개 샘플 
    - Train: 9,600개  
    - Test: 2,400개 
- **언어**: 영어 
- **포맷**: Parquet 또는 Hugging Face Dataset 형태로 제공  
- **필드 구성**:  
  | 필드 이름 | 타입 | 설명 |
  |---|---|---|
  | `context` | 문자열 | 문맥(context) 텍스트, 질문의 근거가 되는 본문 |
  | `question` | 문자열 | 해당 문맥에 대해 묻는 질문 |
  | `answer` | 문자열 | 질문에 대한 정답 (context 기반) |

- **데이터 생성 방식**:
  - `context`는 Falcon RefinedWeb에서 가져온 텍스트를 기반으로 함 
  - `question`과 `answer`은 GPT-4로 생성됨 (즉, 문장 생성 모델을 이용해 질문/답 쌍을 만드는 방식) 



### 필요한 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

sns.set_theme(style="whitegrid")
print("✅ 라이브러리 로드 완료")


### Hugging Face에서 데이터셋 로드

In [ ]:
DATASET_NAME = "neural-bridge/rag-dataset-12000"
print(f"📥 {DATASET_NAME} 데이터셋 다운로드 중...")

dataset = load_dataset(DATASET_NAME, split="train")
print(f"✅ 총 샘플 수: {len(dataset):,}")
print(f"✅ 포함된 필드: {dataset.column_names}")


### 샘플 데이터 미리보기

In [ ]:
NUM_SAMPLES_TO_SHOW = 3

for idx in range(min(NUM_SAMPLES_TO_SHOW, len(dataset))):
    sample = dataset[idx]
    print("━" * 40)
    print(f"🔹 샘플 {idx + 1}")
    for key, value in sample.items():
        preview = str(value)
        if isinstance(value, str) and len(value) > 120:
            preview = value[:120] + "..."
        print(f"  • {key}: {preview}")
print("━" * 40)


### DataFrame으로 변환해 구조 확인

In [ ]:
MAX_ROWS = 1000
subset = dataset.select(range(min(MAX_ROWS, len(dataset))))
df = subset.to_pandas()

print(f"✅ DataFrame 형태: {df.shape[0]}행 × {df.shape[1]}열")
display(df.head())


### 질문과 문서 길이 분포 시각화

In [ ]:
context_column = next((col for col in [
    "positive_ctx",
    "positive_context",
    "context",
    "document",
    "passage",
    "answer_text",
] if col in df.columns), None)

question_lengths = df["question"].astype(str).apply(len)

if context_column is not None:
    context_lengths = df[context_column].astype(str).apply(len)
    lengths = pd.DataFrame({
        "question_length": question_lengths,
        "context_length": context_lengths,
    })
else:
    lengths = pd.DataFrame({"question_length": question_lengths})

num_plots = 2 if context_column is not None else 1
fig, axes = plt.subplots(1, num_plots, figsize=(6 * num_plots, 4))
if num_plots == 1:
    axes = [axes]

def plot_hist(ax, series, title, color):
    sns.histplot(series, ax=ax, bins=30, color=color)
    ax.set_title(title)
    ax.set_xlabel("characters")
    ax.set_ylabel("count")

plot_hist(axes[0], lengths["question_length"], "Question length distribution", "#5A67D8")

if context_column is not None:
    title = f"{context_column} length distribution"
    plot_hist(axes[1], lengths["context_length"], title, "#38B2AC")

plt.tight_layout()
plt.show()


### 질문과 문서 길이 통계 요약

In [ ]:
question_lengths = df["question"].astype(str).apply(len).astype(float)

print("📊 질문 길이 통계")
print(f"  • 평균 길이: {question_lengths.mean():.1f}")
print(f"  • 중앙값 길이: {question_lengths.median():.1f}")
print(f"  • 최대 길이: {question_lengths.max():.0f}")
print(f"  • 중복 질문 수: {(df['question'].astype(str).value_counts() > 1).sum()}")

context_column = next((col for col in [
    "positive_ctx",
    "positive_context",
    "context",
    "document",
    "passage",
    "answer_text",
] if col in df.columns), None)

if context_column is not None:
    context_lengths = df[context_column].astype(str).apply(len).astype(float)
    print("📊 문서 길이 통계")
    print(f"  • 열 이름: {context_column}")
    print(f"  • 평균 길이: {context_lengths.mean():.1f}")
    print(f"  • 중앙값 길이: {context_lengths.median():.1f}")
    print(f"  • 최대 길이: {context_lengths.max():.0f}")


# 질문·문서 길이 통계 확인의 필요성

RAG 파인튜닝을 준비할 때, 데이터셋의 **질문(question)**과 **문서(context)** 길이 분포를 통계적으로 확인하는 것은 매우 중요한 단계다.  
이 과정에서 얻은 통계는 전처리와 모델 학습 전략에 직접적인 영향을 준다.

---

## 1. 토큰 길이 제한 고려
- LLM에는 **최대 입력 토큰 길이** 제한이 존재한다. (예: 4k, 8k, 32k tokens)
- 문서가 평균적으로 길거나 일부 샘플이 지나치게 길다면:
  - 잘라내기(truncation) 과정에서 중요한 정보 손실 위험이 있음
  - 문서를 잘게 나누는 **청킹(chunking)** 전략이 필요해진다

👉 길이 통계는 **청킹 단위 결정**과 **truncation 여부 판단**에 활용된다.

---

## 2. GPU 메모리와 학습 효율 최적화
- 입력 길이가 길수록 GPU 메모리 사용량이 커진다.
- 길이 분포를 보면:
  - 대부분 200~300 tokens → 큰 배치(batch size) 가능
  - 평균 1000 tokens 이상 → 배치 크기 줄여야 함 → 학습 속도 저하

👉 통계 기반으로 **batch size, gradient accumulation** 같은 하이퍼파라미터를 조정할 수 있다.

---

## 3. 데이터 품질 점검
- **너무 짧은 질문/문서**: 정보 부족으로 학습에 noise 발생
- **지나치게 긴 문서**: 불필요한 문맥이 포함되어 모델 혼란 초래

👉 통계를 통해 **필터링 규칙**을 세울 수 있다.  
예: 특정 길이 미만 샘플 제거, 너무 긴 샘플 분할 등.

---

## 4. RAG 파이프라인 설계와 연결
- RAG는 보통 검색된 문서 `top-k`개를 모델 입력에 넣는다.
- 문서 길이 분포를 미리 알면:
  - 한 번에 넣을 수 있는 문서 개수 결정 가능 (예: top-3 vs top-5)
  - 모델 입력 제한에 따라 `k` 값 조정

👉 예: 한 문서 평균 500 tokens → top-5 문서를 넣으면 약 2500 tokens 필요.

---

## 5. 평가와 분석 기준 제공
- 길이 분포를 기준으로 성능 차이를 분석할 수 있다.
  - 짧은 context vs 긴 context → 정확도 차이 비교
  - 질문 길이가 짧을 때 검색 성능이 더 떨어지는지 검증

👉 통계 없이는 이런 **세부 분석**이 불가능하다.

---

## ✅ 결론
질문·문서 길이 통계는 단순한 탐색이 아니라,  
- **청킹/필터링 전략**  
- **batch size 및 학습 효율**  
- **검색된 문서 top-k 설계**  
- **성능 평가 기준 설정**  

등 전처리와 학습 전략의 핵심 근거가 된다.


### 다음 단계
- `01_data_preprocessing_and_validation.ipynb`에서 이 데이터를 RAFT 템플릿에 맞춰 가공합니다.
- 전처리 결과는 Google Drive에 저장해 두고 이후 노트북에서 `load_from_disk`로 불러오세요.
